## Traditional Machine Learning

In this notebook we'll be approaching our classification problem of whether a tweet references an ongoing disaster with a few traditional classifiers. We'll use the TF-IDF we produced in our data wrangling step as the input to our predictive models, and train and predict on the provided labels. After we have also analyzed the accuracy of a neural network based classifier in another notebook we will be able to pick one superior final model to move forward with.

In [2]:
# First load our input and outputs and produce our train and test set
import pandas as pd
import scipy.sparse
from sklearn.model_selection import train_test_split

X = scipy.sparse.load_npz('../data/tweets-tf-idf.npz')

tweet_df = pd.read_csv('../data/processed_kaggle_training.csv')
y = tweet_df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

### Random Forest

Due to the high dimensionality of our TF-IDF input, standard logistic regression is unlikely to perform well. Certain kernel functions may allow a functional model to be produced with SVM, but a non linear approach is likely to be the way to go, so in this notebook we will first train a Random Forest model with sklearn, then the popular gradient boosted model with the machine learning library XGBoost, and then for one more paradigm commonly applied in NLP we will train a naive bayes classifier.

We will utilize the sklearn impletmentation of random forest. In the random forest model, the initial data set is remade repeatedly at random through bagging, with a new decision tree being trained on each bagging. Afterwards predictions are made by ensemble voting over the array of decision trees, generally producing a much better performing predictor than one decision tree on its own could obtain. We'll do some grid searching to pick an optimal number of estimators, and features for each tree to consider, but will be limited by the size of our input space and compute power, from traversing a greater set of hyperparameters.

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

rfc = RandomForestClassifier(random_state=42)

# with an input dimension of ~7000 words, tuning on more than 2
# params is not feasible
parameters = {
    'n_estimators': [100, 200, 400, 750, 1000],
    'max_features': ['auto', 'sqrt', 'log2']
}
gcv_rfc = GridSearchCV(rfc, parameters, cv=5)

gcv_rfc.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [100, 200, 400, 750, 1000], 'max_features': ['auto', 'sqrt', 'log2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [26]:
# Our best model
gcv_rfc.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='log2', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [23]:
# And the accuracy of our best RF model is
gcv_rfc.best_estimator_.score(X_test, y_test)

0.7898883782009193

### XGBoost Classification
XGBoost classifiers are largely replacing random forest in practice. They represent a specific implementation of the gradient boost algorithm which still collects an ensemble of decision trees like random forest, but instead of building a forest of trees at random and somewhat concurrently, trees are added to the ensemble sequentially, with each new tree trained on the residuals of the model up to that point, and then the prediction of the new tree is scaled by a learning_rate parameter, and added to the total predictive model up to that point. XGBoost in particular, along with some implementation optimization, adds a complexity weight to each tree over just minimizing the given objective function, and so performs some regularization on the input parameters. Altogether this generally forms a better predictive model then base random forest achieves.

In [13]:
from xgboost.sklearn import XGBClassifier

xgbc = XGBClassifier(
 learning_rate = 0.1,
 n_estimators = 1000,
 objective= 'binary:logistic',
 seed=42)

# with more time we might also tune the above hyperparams, but
# we choose the following hyperparams to search on as typically the most
# significant
params_xgbc = {
 'max_depth': [5, 10, 15],
 'min_child_weight': [1, 3, 6],
 'subsample': [0.6, 0.8, 1]
}

gcv_xgbc = GridSearchCV(xgbc, params_xgbc, cv=5)
gcv_xgbc.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
       colsample_bynode=None, colsample_bytree=None, gamma=None,
       gpu_id=None, importance_type='gain', interaction_constraints=None,
       learning_rate=0.1, max_delta_step=None, max_depth=None,
       min_child_we...t=None, seed=42, subsample=None,
       tree_method=None, validate_parameters=False, verbosity=None),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'max_depth': [5, 10, 15], 'min_child_weight': [1, 3, 6], 'subsample': [0.6, 0.8, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [25]:
# Our Best Model
gcv_xgbc.best_estimator_

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints=None,
       learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=nan, monotone_constraints=None,
       n_estimators=1000, n_jobs=0, num_parallel_tree=1,
       objective='binary:logistic', random_state=42, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=42, subsample=1,
       tree_method=None, validate_parameters=False, verbosity=None)

In [22]:
# The accuracy of our best model
gcv_xgbc.best_estimator_.score(X_test, y_test)

0.7846355876559422

### Naive Bayes

Naive bayes classifiers are an often used classifier in NLP as their predictive performance can be superior on the sparse input data sets (and our tweets are particularly sparse input vectors). They are used in the same way that we have made use of the random forest above, to classify the sentiment of texts provided as bag of words (often in the TF-IDF format as we have obtained). But instead of training an ensemble, naive bayes will attempt to iteratively arrive at the probabilities of documents (tweets) belonging to a particular class (disaster or not disaster) given particular words appearing in that document. Afterwards there will be one set of feature importances for each token by which we can calculate what class an unseen tweet likely belongs to.

In [19]:
from sklearn.naive_bayes import GaussianNB

# There isn't actually anything to tune with GaussianNB
gnb = GaussianNB()

gnb.fit(X_train.toarray(), y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [24]:
gnb.score(X_test.toarray(), y_test)

0.6677609980302035

But it seems this is our weakest performer and is perhaps why XGBoost is taking the machine learning world by storm.